<a href="https://colab.research.google.com/github/sklearner26/signlanguage/blob/main/ISLR_streamlit_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit
!pip install pyngrok
!pip install streamlit_option_menu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 114.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.3/829.3 kB 8.1 MB/s eta 0:00:00


In [ ]:
!pip install mediapipe

  Using cached protobuf-4.25.5-cp37-abi3-manylinux2014_x86_64.whl.metadata (541 bytes)
Using cached protobuf-4.25.5-cp37-abi3-manylinux2014_x86_64.whl (294 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.16.1 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.5 which is incompatible.


In [9]:
!pip install textdistance

In [11]:
txt='''
i want water
i want food
i want help
carry it
give me
my name is Shreyansh Kesarwani
my name is sarika yadav
my name is ankit singh
my name is harshit
my name is prince
my name is jitin
i want help
i have emergency situation
call police
call me
hello guys
hello mam
good afternoon
good bye
good evening
hello my name is shreyansh kesarwani
i study in university of allahabad
i learned data science
i have expertise in computer vision
i am data science student
i am very good in deep learning
i want to help you
i live in prayagraj
i live in delhi
i live in lucknow
i live in delhi
how are you
i think you are right
you have to change yourself
i want i am in danger
please help me
help me
give me
i need help
hello everyone my name is
this is indian sign language app
i love to play cricket
sport
accept agree deny
no
not this that bye
football volleyball
sign language
indian recognition
detection which where why
used next word prediction predict
decision making
make items
products
'''

In [ ]:
video_address='https://cdn.pixabay.com/video/2023/11/11/188742-883619742_tiny.mp4'

In [36]:
%%writefile app2.py
import streamlit as st
import numpy as np
import pandas as pd
import textdistance
import re
from collections import Counter
import os

# Function to load and repeat each word 50 times
def load_and_repeat_words(file_path):
    words = []
    with open(file_path, encoding='utf-8') as f:
        data = f.read()
        data = data.lower()  # Convert to lowercase for uniformity
        word_list = re.findall(r'\w+', data)  # Collect all words
        # Repeat each word 50 times and append to the words list
        words += word_list * 50  # Repeat each word 50 times
    return words

# Load and process the first text file
words1 = load_and_repeat_words('/content/words.txt')

# Combine the words from both files
words = words1

# Remove duplicates to get a unique set of words
V = set(words)

# Count frequency of each word
word_freq_dict = Counter(words)

# Calculate the relative probability of each word
N = sum(word_freq_dict.values())
prob = {w: word_freq_dict[w] / N for w in word_freq_dict}

# Function to predict the next words starting with the input prefix
def predict_next_words(prefix, top_n=5):
    """
    Given a word prefix, predict the top_n most likely next words based on frequency and similarity,
    excluding the prefix itself.
    """
    # Find words that start with the given prefix, excluding the prefix itself
    matching_words = [word for word in word_freq_dict if word.startswith(prefix) and word != prefix]

    if not matching_words:
        return f"No words found starting with '{prefix}'"

    # Calculate Jaccard similarity for each matching word (similarity can be based on edit distance)
    similarity = []
    for word in matching_words:
        sim = 1 - textdistance.Jaccard(qval=2).distance(prefix, word)  # Using Jaccard distance with qval=2
        similarity.append((word, sim, prob[word]))  # Include word, similarity score, and probability

    # Sort by similarity (descending) and probability (descending), and select the top_n words
    sorted_similarities = sorted(similarity, key=lambda x: (x[1], x[2]), reverse=True)

    # Get the top_n words
    top_predictions = sorted_similarities[:top_n]

    return top_predictions

# Function to show the homepage with title and video
def show_home_page():
    st.markdown(
        """
        <style>
        .title {
            color: red;
            font-size: 2.5em;
            font-weight: bold;
            text-align: center;
            margin-top: 20px;
        }
        .video-container {
            width: 80%;
            margin: 20px auto;
            display: block;
            text-align: center;
            position: relative;
        }
        </style>
        """, unsafe_allow_html=True
    )

    # Title
    st.markdown('<div class="title">Welcome to Indian Sign Language App</div>', unsafe_allow_html=True)

    # Add the video (make sure it is appropriately sized to avoid excess space)
    st.markdown("""
    <div class="video-container">
        <video width="100%" autoplay loop muted>
            <source src="https://cdn.pixabay.com/video/2023/11/11/188742-883619742_tiny.mp4" type="video/mp4"> <!-- Replace with your video URL -->
            Your browser does not support the video tag.
        </video>
    </div>
    """, unsafe_allow_html=True)

# Function to display the NLP page
def show_nlp_page():
    st.markdown('<h1 style="color:red;">NLP - Next Word Prediction</h1>', unsafe_allow_html=True)
    st.write('Enter a prefix and click "Predict Next Words" to see the most likely next words.')

    # Input field for the user to enter a prefix
    prefix_input = st.text_input('Enter word prefix', '')

    # Button to trigger the prediction
    if st.button('Predict Next Words'):
        if prefix_input:
            # Get the top 5 words for the given prefix
            predictions = predict_next_words(prefix_input, top_n=5)

            # Display the predictions in a table or list format
            if isinstance(predictions, list):
                prediction_df = pd.DataFrame(predictions, columns=['Word', 'Similarity', 'Probability'])
                st.write(prediction_df)
            else:
                st.write(predictions)
        else:
            st.write("Please enter a prefix.")

# Function to display the Sign Gesture Prediction page
def show_sign_gesture_page():
    st.markdown('<h1 style="color:red;">Sign Gesture Prediction</h1>', unsafe_allow_html=True)
    st.write('This page will show sign gesture prediction functionality. Currently, this is a placeholder.')

# Function to show the textual data (words)
def show_textual_data_page():
    st.markdown('<h1 style="color:red;">Textual Data - Words from File</h1>', unsafe_allow_html=True)

    # Display words in tabular form
    df = pd.DataFrame(words, columns=["Unique Words"])
    st.write("**Unique Words** from the dataset:", df)

    # Provide an option to add new words
    new_word = st.text_input("Enter a word to add:", "")
    if st.button("Add Word"):
        if new_word:
            # Add the new word to the session (only for the current session)
            words.append(new_word.lower())  # Add lowercase version of the word
            st.success(f"Word '{new_word}' added!")
        else:
            st.warning("Please enter a word before adding.")

    # Show updated table with the added words
    st.write("Updated List of Words:")
    df_updated = pd.DataFrame(list(set(words)), columns=["Unique Words"])  # Ensure words remain unique
    st.write(df_updated)

# Function to display the sign data (26 images for A-Z)
def show_sign_data_page():
    st.markdown('<h1 style="color:red;">Sign Gesture Data - A-Z</h1>', unsafe_allow_html=True)

    # Create 26 images, one for each letter of the alphabet
    # Assume images are named 'a.jpg', 'b.jpg', ..., 'z.jpg'
    sign_images = [f"sign_images/{chr(65+i)}.jpg" for i in range(26)]  # Replace with your path to images

    # Display images in 4 columns per row
    cols = st.columns(4)
    for i, col in enumerate(cols):
        start_idx = i * 6
        end_idx = min(start_idx + 6, 26)
        for j in range(start_idx, end_idx):
            col.image(sign_images[j], caption=chr(65+j), use_column_width=True)

# Streamlit layout with sidebar navigation
st.sidebar.title("Indian Sign Language App")
page = st.sidebar.selectbox("Select a Page", ["Home", "NLP - Next Word Prediction", "Textual Data", "Sign Data", "Sign Gesture Prediction"])

# Display corresponding page based on user selection
if page == "Home":
    # Show home page with title and video
    show_home_page()

elif page == "NLP - Next Word Prediction":
    # Show the NLP prediction page
    show_nlp_page()

elif page == "Textual Data":
    # Show textual data page
    show_textual_data_page()

elif page == "Sign Data":
    # Show sign data page (A-Z images)
    show_sign_data_page()

elif page == "Sign Gesture Prediction":
    # Show the sign gesture prediction page
    show_sign_gesture_page()



Overwriting app2.py


In [69]:
%%writefile app2.py
import streamlit as st
import numpy as np
import pandas as pd
import textdistance
import re
from sign import process_sign_gesture
from collections import Counter
import os

# Function to display the Sign Gesture Prediction page
def show_sign_gesture_page():
    st.markdown('<h1 style="color:red;">Sign Gesture Prediction</h1>', unsafe_allow_html=True)
    st.write('This page will show sign gesture prediction functionality. Currently, this is a placeholder.')
    process_sign_gesture()


def show_sign_data_page():
    st.markdown('<h1 style="color:red;">Sign Gesture Data - A-Z</h1>', unsafe_allow_html=True)

    # Create 26 images, one for each letter of the alphabet
    # Assume images are named 'a.jpg', 'b.jpg', ..., 'z.jpg'
    sign_images = [f"sign_images/{chr(65+i)}.jpg" for i in range(26)]  # Replace with your path to images

    # Display images in 4 columns per row
    cols = st.columns(4)

    # Loop over the images and place them in the columns
    for i in range(0, 26, 4):  # Loop over the images in chunks of 4
        for j, col in enumerate(cols):
            # Calculate the index of the current image
            image_idx = i + j
            if image_idx < 26:  # Ensure we don't go beyond 26 images
                image_path = sign_images[image_idx]

                # Check if the image exists at the specified path
                if os.path.exists(image_path):
                    # Display the image and label it with the letter
                    col.image(image_path, caption=chr(65 + image_idx), use_container_width=True)
                else:
                    # If image doesn't exist, display a message
                    col.write(f"Image for '{chr(65 + image_idx)}' not found at path: {image_path}")


def load_and_repeat_words(file_path):
    words = []
    with open(file_path, encoding='utf-8') as f:
        data = f.read()
        data = data.lower()  # Convert to lowercase for uniformity
        word_list = re.findall(r'\w+', data)  # Collect all words
        words += word_list * 50  # Repeat each word 50 times
    return words

# Load and process the first text file
words1 = load_and_repeat_words('/content/words.txt')

# Combine the words and make sure to use only unique words
words = list(set(words1))  # Remove duplicates to get a unique set of words

# Load or create the temp file that will hold additional words for the current session
temp_file = 'temp.txt'

# Function to save the updated word list to the temp file
def save_temp_data(word_list):
    with open(temp_file, 'w', encoding='utf-8') as f:
        for word in word_list:
            f.write(f"{word}\n")

# Function to load the temporary data from the temp file
def load_temp_data():
    if os.path.exists(temp_file):
        with open(temp_file, 'r', encoding='utf-8') as f:
            temp_words = f.read().splitlines()
            return temp_words
    return []

# Function to show the textual data (words)
def show_textual_data_page():
    st.markdown('<h1 style="color:red;">Textual Data - Words from File</h1>', unsafe_allow_html=True)

    # Combine loaded words with temp words (session data)
    temp_words = load_temp_data()
    all_words = list(set(words + temp_words))  # Ensure words remain unique by combining both sources

    # Display words in a grid (5 words per row)
    columns = st.columns(5)
    for i, word in enumerate(all_words):
        col = columns[i % 5]
        col.write(word)  # Display word in the respective column

    # Provide an option to add new words
    new_word = st.text_input("Enter a word to add:", "")
    if st.button("Add Word"):
        if new_word:
            # Add the new word to the session (only for the current session)
            new_word_lower = new_word.lower()  # Add lowercase version of the word
            temp_words.append(new_word_lower)
            save_temp_data(temp_words)  # Save to temp.txt file for session persistence
            st.success(f"Word '{new_word}' added!")
        else:
            st.warning("Please enter a word before adding.")

# Function to show the NLP page
def show_nlp_page():
    st.markdown('<h1 style="color:red;">NLP - Next Word Prediction</h1>', unsafe_allow_html=True)
    st.write('Enter a prefix and click "Predict Next Words" to see the most likely next words.')

    # Combine loaded words with temp words (session data)
    temp_words = load_temp_data()
    all_words = list(set(words + temp_words))  # Ensure words remain unique

    # Input field for the user to enter a prefix
    prefix_input = st.text_input('Enter word prefix', '')

    # Button to trigger the prediction
    if st.button('Predict Next Words'):
        if prefix_input:
            # Get the top 5 words for the given prefix from the combined list
            predictions = predict_next_words(prefix_input, all_words, top_n=5)

            # Display the predictions in a table or list format
            if isinstance(predictions, list):
                prediction_df = pd.DataFrame(predictions, columns=['Word', 'Similarity', 'Probability'])
                st.write(prediction_df)
            else:
                st.write(predictions)
        else:
            st.write("Please enter a prefix.")

# Function to predict the next words based on input prefix
def predict_next_words(prefix, word_list, top_n=5):
    """
    Given a word prefix, predict the top_n most likely next words based on frequency and similarity,
    excluding the prefix itself.
    """
    # Count frequency of each word
    word_freq_dict = Counter(word_list)

    # Calculate the relative probability of each word
    N = sum(word_freq_dict.values())
    prob = {w: word_freq_dict[w] / N for w in word_freq_dict}

    # Find words that start with the given prefix, excluding the prefix itself
    matching_words = [word for word in word_freq_dict if word.startswith(prefix) and word != prefix]

    if not matching_words:
        return f"No words found starting with '{prefix}'"

    # Calculate Jaccard similarity for each matching word (similarity can be based on edit distance)
    similarity = []
    for word in matching_words:
        sim = 1 - textdistance.Jaccard(qval=2).distance(prefix, word)  # Using Jaccard distance with qval=2
        similarity.append((word, sim, prob[word]))  # Include word, similarity score, and probability

    # Sort by similarity (descending) and probability (descending), and select the top_n words
    sorted_similarities = sorted(similarity, key=lambda x: (x[1], x[2]), reverse=True)

    # Get the top_n words
    top_predictions = sorted_similarities[:top_n]

    return top_predictions

# Function to show the homepage with title and video
def show_home_page():
    st.markdown(
        """
        <style>
        .title {
            color: red;
            font-size: 2.5em;
            font-weight: bold;
            text-align: center;
            margin-top: 20px;
        }
        .video-container {
            width: 80%;
            margin: 20px auto;
            display: block;
            text-align: center;
            position: relative;
        }
        </style>
        """, unsafe_allow_html=True
    )

    # Title
    st.markdown('<div class="title">Welcome to Indian Sign Language App</div>', unsafe_allow_html=True)

    # Add the video (make sure it is appropriately sized to avoid excess space)
    st.markdown("""
    <div class="video-container">
        <video width="100%" autoplay loop muted>
            <source src="https://cdn.pixabay.com/video/2023/11/11/188742-883619742_tiny.mp4" type="video/mp4"> <!-- Replace with your video URL -->
            Your browser does not support the video tag.
        </video>
    </div>
    """, unsafe_allow_html=True)


# Streamlit layout with sidebar navigation
st.sidebar.title("Indian Sign Language App")
page = st.sidebar.selectbox("Select a Page", ["Home", "NLP - Next Word Prediction", "Textual Data", "Sign Data", "Sign Gesture Prediction"])

# Display corresponding page based on user selection
if page == "Home":
    # Show home page with title and video
    show_home_page()

elif page == "NLP - Next Word Prediction":
    # Show the NLP prediction page
    show_nlp_page()

elif page == "Textual Data":
    # Show textual data page
    show_textual_data_page()

elif page == "Sign Data":
    # Show sign data page (A-Z images)
    show_sign_data_page()

elif page == "Sign Gesture Prediction":
    # Show the sign gesture prediction page
    show_sign_gesture_page()

Overwriting app2.py


In [66]:
import os
from threading import Thread
from pyngrok import ngrok

In [72]:
ngrok.kill()

In [73]:
ngrok.set_auth_token('2eAyXiyxF7ABzEMlJbIX2wphIEJ_4H5QaGTrJ35jghea6ADs4')

In [74]:
!ngrok config add-authtoken '2eAyXiyxF7ABzEMlJbIX2wphIEJ_4H5QaGTrJ35jghea6ADs4'

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [75]:
def run_streamlit():
  os.system('streamlit run /content/app2.py --server.port 8000')

In [76]:
thread=Thread(target=run_streamlit)
thread.start()

In [77]:
public_url=ngrok.connect(addr='8000',proto='http',bind_tls=True)
print("your streamlit app is live at:",public_url)

your streamlit app is live at: NgrokTunnel: "https://b06e-34-106-122-137.ngrok-free.app" -> "http://localhost:8000"


In [ ]:
!ngrok http --domain=puma-humorous-cat.ngrok-free.app

In [ ]:
!ssh -R puma-humorous-cat.ngrok-free.app:80:https://550e-34-141-137-68.ngrok-free.app/ serveo.net

In [ ]:
!pip show tensorflow


Name: tensorflow
Version: 2.17.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, requests, setuptools, six, tensorboard, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: dopamine_rl, tf_keras


In [ ]:
import smtplib

EMAIL_ADDRESS = "hello@gmail.com"  # Replace with your email
EMAIL_PASSWORD = "12345678"  # Replace with your app password

def send_test_email():
    try:
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()
            server.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
            server.sendmail(EMAIL_ADDRESS, EMAIL_ADDRESS, "Test email from Python!")
            print("Email sent successfully!")
    except Exception as e:
        print(f"Error: {e}")

send_test_email()


Error: (535, b'5.7.8 Username and Password not accepted. For more information, go to\n5.7.8  https://support.google.com/mail/?p=BadCredentials a1e0cc1a2514c-85655796506sm1446348241.33 - gsmtp')


In [ ]:
import shutil
shutil.rmtree('/content/utils')

# **Implementing django app**


/content


In [ ]:
!pip install django

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 49.2 MB/s eta 0:00:00


In [ ]:
!django-admin startproject islr

In [ ]:
%cd islr/

/content/islr


In [ ]:
!python manage.py startapp islr_app

In [ ]:
%cd islr_app/

/content/islr/islr_app


In [ ]:
%%writefile views.py
from django.shortcuts import render
from django.http import HttpResponse
# Create your views here.
def display(request):
  return HttpResponse('Hello world!')

Overwriting views.py


In [ ]:
%cd /content/islr/

/content/islr


In [ ]:
!python manage.py migrate

Operations to perform:
  Apply all migrations: admin, auth, contenttypes, sessions
Running migrations:
  Applying contenttypes.0001_initial... OK
  Applying auth.0001_initial... OK
  Applying admin.0001_initial... OK
  Applying admin.0002_logentry_remove_auto_add... OK
  Applying admin.0003_logentry_add_action_flag_choices... OK
  Applying contenttypes.0002_remove_content_type_name... OK
  Applying auth.0002_alter_permission_name_max_length... OK
  Applying auth.0003_alter_user_email_max_length... OK
  Applying auth.0004_alter_user_username_opts... OK
  Applying auth.0005_alter_user_last_login_null... OK
  Applying auth.0006_require_contenttypes_0002... OK
  Applying auth.0007_alter_validators_add_error_messages... OK
  Applying auth.0008_alter_user_username_max_length... OK
  Applying auth.0009_alter_user_last_name_max_length... OK
  Applying auth.0010_alter_group_name_max_length... OK
  Applying auth.0011_update_proxy_permissions... OK
  Applying auth.0012_alter_user_first_name_max_l

In [ ]:
def run_django():
  os.system('python manage.py --server.port 7000')

In [ ]:
ngrok.connect(7000)

PyngrokNgrokHTTPError: ngrok client exception, API returned 502: {"error_code":103,"status_code":502,"msg":"failed to start tunnel","details":{"err":"failed to start tunnel: Your account may not run more than 3 tunnels over a single ngrok agent session.\nThe tunnels already running on this session are:\ntn_2oesozNa5NPaXTtvVzt1nsWOZlD, tn_2of3ZTqovE9kepUuxGKSpx8F703, tn_2of3igAdja5FngDCTdidauJbB3j\n\r\n\r\nERR_NGROK_324\r\n"}}


In [ ]:
thread=Thread(target=run_django)
thread.start()

In [ ]:
public_url=ngrok.connect(addr='7000',proto='http',bind_tls=True)
print("your django app is live at:",public_url)

PyngrokNgrokHTTPError: ngrok client exception, API returned 502: {"error_code":103,"status_code":502,"msg":"failed to start tunnel","details":{"err":"failed to start tunnel: Your account may not run more than 3 tunnels over a single ngrok agent session.\nThe tunnels already running on this session are:\ntn_2of3igAdja5FngDCTdidauJbB3j, tn_2oesozNa5NPaXTtvVzt1nsWOZlD, tn_2of3ZTqovE9kepUuxGKSpx8F703\n\r\n\r\nERR_NGROK_324\r\n"}}


In [ ]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(8000)"))

https://phrnf33euus-496ff2e9c6d22116-8000-colab.googleusercontent.com/


In [ ]:
!python manage.py runserver 7000

Watching for file changes with StatReloader
Performing system checks...

System check identified no issues (0 silenced).
November 10, 2024 - 14:02:10
Django version 5.1.3, using settings 'islr.settings'
Starting development server at http://127.0.0.1:7000/
Quit the server with CONTROL-C.

[10/Nov/2024 14:03:45] "GET / HTTP/1.1" 200 12
Not Found: /favicon.ico
[10/Nov/2024 14:03:45] "GET /favicon.ico HTTP/1.1" 404 2383
[10/Nov/2024 14:05:21] "GET / HTTP/1.1" 200 12
